<a href="https://colab.research.google.com/github/asm916/image-processing/blob/main/Object_cover_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 12.9 MB/s 
     |████████████████████████████████| 6.3 MB 50.9 MB/s 
     |████████████████████████████████| 15.4 MB 49.4 MB/s 


In [ ]:
pip install gdal

In [ ]:
import numpy as np
import gdal
import ogr
from skimage import exposure
from skimage.segmentation import quickshift
from skimage.segmentation import slic
import geopandas as gpd
import numpy as np
import time
 
naip_fn = '/content/stimage.png'
 
driverTiff = gdal.GetDriverByName('GTiff')
print("driverTiff", driverTiff)

naip_ds = gdal.Open(naip_fn)

nbands = naip_ds.RasterCount
band_data = []

for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)
band_data = np.dstack(band_data)
img = exposure.rescale_intensity(band_data)

# do segmentation, different options with quickshift and slic (only use one of the next two lines)
segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
print(segments)
#print('segments complete', time.time() - seg_start)
 
# save segments to raster
segments_fn = '/segments.tif'
print("------------------------------------")
print("RasterXSize:", naip_ds.RasterXSize)
print("RasterYSize:", naip_ds.RasterYSize)
print("gdal.GDT_Float32", gdal.GDT_Float32)
print("------------------------------------")
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                1, gdal.GDT_Float32)
print("segments_ds", segments_ds)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None

driverTiff <osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f9c0f3ffb40> >
[[ 29  29  29 ...  44  44  44]
 [ 29  29  29 ...  44  44  44]
 [ 29  29  29 ...  44  44  44]
 ...
 [862 862 862 ... 811 811 811]
 [862 862 862 ... 811 811 811]
 [862 862 862 ... 811 811 811]]
------------------------------------
RasterXSize: 788
RasterYSize: 470
gdal.GDT_Float32 6
------------------------------------
segments_ds <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f9c182184b0> >


In [ ]:
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features

In [ ]:
import scipy
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    objects.append(object_features)
    object_ids.append(id)

In [ ]:
# read shapefile to geopandas geodataframe
gdf = gpd.read_file('/truth_data.shp')
# get names of land cover classes/labels
class_names = gdf['label'].unique()
# create a unique id (integer) for each land cover class/label
class_ids = np.arange(class_names.size) + 1
# create a pandas data frame of the labels and ids and save to csv
df = pd.DataFrame({'label': class_names, 'id': class_ids})
df.to_csv('C:/temp/naip/class_lookup.csv')
# add a new column to geodatafame with the id for each class/label
gdf['id'] = gdf['label'].map(dict(zip(class_names, class_ids)))
 
# split the truth data into training and test data sets and save each to a new shapefile
gdf_train = gdf.sample(frac=0.7)  # 70% of observations assigned to training data (30% to test data)
gdf_test = gdf.drop(gdf_train.index)
# save training and test data to shapefiles
gdf_train.to_file('path/to/save/train_data.shp')
gdf_test.to_file('path/to/save/test_data.shp')


DriverError: ignored